In [2]:
import pandas as pd
import matplotlib.pyplot as plt  
import numpy as np 
import seaborn as sns 

In [24]:
data_lite = pd.read_csv('SBAnational.csv')

# obtenir le nombre de lignes et de colonnes de votre dataframe
nombre_lignes, nombre_colonnes = data_lite.shape

print("Nombre de lignes :", nombre_lignes)
print("Nombre de colonnes :", nombre_colonnes)


/tmp/ipykernel_40092/752727223.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  data_lite = pd.read_csv('SBAnational.csv')


Nombre de lignes : 899164
Nombre de colonnes : 27


##### On crée un nouveau dataframe où les colonnes qui sont inutiles pour les prédictions sont supprimées ainsi que les variables suspectées de faire du dataleaking.
En faisant un randomforest sur les données brutes, on obtient des scores trés proches de 1 avec les variables: 'DisbursementDate', 'BalanceGross', 'LoanNr_ChkDgt', 'ChgOffDate', 'DisbursementGross', 'ChgOffPrinGr'.

In [4]:
#data_lite = data.drop(['DisbursementDate', 'BalanceGross', 'LoanNr_ChkDgt', 'ChgOffDate', 'DisbursementGross', 'ChgOffPrinGr'], axis=1)
#data_lite.head(2)

#####  Vérifier les types des variables, les valeurs manquantes ou suspectes.
On vérifie les valeurs uniques pour chaque variable du dataframe ainsi que les 'missing values'.

In [5]:
print(data_lite.isnull().sum())

data_lite.dtypes


LoanNr_ChkDgt             0
Name                     14
City                     30
State                    14
Zip                       0
Bank                   1559
BankState              1566
NAICS                     0
ApprovalDate              0
ApprovalFY                0
Term                      0
NoEmp                     0
NewExist                136
CreateJob                 0
RetainedJob               0
FranchiseCode             0
UrbanRural                0
RevLineCr              4528
LowDoc                 2582
ChgOffDate           736465
DisbursementDate       2368
DisbursementGross         0
BalanceGross              0
MIS_Status             1997
ChgOffPrinGr              0
GrAppv                    0
SBA_Appv                  0
dtype: int64


LoanNr_ChkDgt          int64
Name                  object
City                  object
State                 object
Zip                    int64
Bank                  object
BankState             object
NAICS                  int64
ApprovalDate          object
ApprovalFY            object
Term                   int64
NoEmp                  int64
NewExist             float64
CreateJob              int64
RetainedJob            int64
FranchiseCode          int64
UrbanRural             int64
RevLineCr             object
LowDoc                object
ChgOffDate            object
DisbursementDate      object
DisbursementGross     object
BalanceGross          object
MIS_Status            object
ChgOffPrinGr          object
GrAppv                object
SBA_Appv              object
dtype: object

#### Vérifie l'existence de doublons et les supprimer le cas échéant

In [6]:
#doublons sur l'ensemble des variables
doublons = data_lite[data_lite.duplicated(keep='first')]
print (doublons)


# #supprime les doublons trouvés et vérifie qu'ils sont absents
data_lite = data_lite.drop(doublons.index)
print(doublons)


Empty DataFrame
Columns: [LoanNr_ChkDgt, Name, City, State, Zip, Bank, BankState, NAICS, ApprovalDate, ApprovalFY, Term, NoEmp, NewExist, CreateJob, RetainedJob, FranchiseCode, UrbanRural, RevLineCr, LowDoc, ChgOffDate, DisbursementDate, DisbursementGross, BalanceGross, MIS_Status, ChgOffPrinGr, GrAppv, SBA_Appv]
Index: []

[0 rows x 27 columns]
Empty DataFrame
Columns: [LoanNr_ChkDgt, Name, City, State, Zip, Bank, BankState, NAICS, ApprovalDate, ApprovalFY, Term, NoEmp, NewExist, CreateJob, RetainedJob, FranchiseCode, UrbanRural, RevLineCr, LowDoc, ChgOffDate, DisbursementDate, DisbursementGross, BalanceGross, MIS_Status, ChgOffPrinGr, GrAppv, SBA_Appv]
Index: []

[0 rows x 27 columns]


#### Vérifier valeurs manquantes dans target pour les supprimer

In [7]:
data_lite['MIS_Status'].isnull().sum()

data_lite.dropna(subset=['MIS_Status'], inplace=True)

print(data_lite['MIS_Status'].unique())


['P I F' 'CHGOFF']


On transforme les dates en type 'DateTime'

In [8]:
data_lite['ApprovalDate'] = pd.to_datetime(data_lite['ApprovalDate'], dayfirst= False) #pour convertir des données en date

pd.options.display.date_dayfirst = True
data_lite['ApprovalDate'] = data_lite['ApprovalDate'].dt.strftime('%d/%m/%Y')  #pour formater les dates déjà converties dans un format specifique
data_lite

#retirer A de 1976A dans les cases concernées
data_lite['ApprovalFY'] = data_lite['ApprovalFY'].str.replace('A', '')
data_lite['ApprovalFY'].dtypes #object

data_lite.head(2)
data_lite.dtypes

LoanNr_ChkDgt          int64
Name                  object
City                  object
State                 object
Zip                    int64
Bank                  object
BankState             object
NAICS                  int64
ApprovalDate          object
ApprovalFY            object
Term                   int64
NoEmp                  int64
NewExist             float64
CreateJob              int64
RetainedJob            int64
FranchiseCode          int64
UrbanRural             int64
RevLineCr             object
LowDoc                object
ChgOffDate            object
DisbursementDate      object
DisbursementGross     object
BalanceGross          object
MIS_Status            object
ChgOffPrinGr          object
GrAppv                object
SBA_Appv              object
dtype: object

Vérifier les valeurs uniques des colonnes. Si les valeurs différent des modalités officielles, on les remplace par NaN.

In [9]:
data_lite['MIS_Status'].unique() 

# on supprime les valeurs manquantes de la target
data_lite.dropna(subset=['MIS_Status'], inplace=True)

# encodage de la target en variables numérique
data_lite['MIS_Status'] = data_lite['MIS_Status'].map({'P I F': 0, 'CHGOFF': 1})
data_lite.MIS_Status.dtype

dtype('int64')

In [10]:
data_lite['RevLineCr'].unique() # renvoie:array(['N', '0', 'Y', 'T',......, 'Q']

#cree variable qui stocke les valeurs anormales de revlinecr
valeurs_odd = [ 'NaN', 'nan','0', 'T','`', ',', '1', 'C', '3', '2', 'R', '7','A', '5', '.', '4', '-', 'Q']

#reamplace valeurs etranges par Nan
data_lite['RevLineCr'] = data_lite['RevLineCr'].replace(valeurs_odd, np.nan)

#verifie les valeurs uniques de revlinecr
data_lite['RevLineCr'].unique()

array(['N', nan, 'Y'], dtype=object)

In [11]:
data_lite['Term'].unique() #valeurs numériques 

array([ 84,  60, 180, 240, 120,  45, 297, 162,  12, 300,  87, 114, 144,
       126,  83, 102,  80, 137,  42,  96, 167,   7,  36,  37,  26, 264,
        72,  24,   5,  54,  66, 161,  71,   4,  93, 288, 108,  10,  13,
        90,  19,  16,   3,  27, 149,  41, 246,  18,  57, 104,  82, 298,
        14,  61, 127,  58,  44,  32,  85,  48,  31, 112,  38,  73,  47,
        11, 134,  15,  79,  53,  39,   6, 255,  55, 133,  95,  35,  59,
        62,  68, 123,  46,  70, 138,  40,  52,  25,  65,  91,   1,  74,
        49, 103,  77,  86,  63,  56,  22,   0,  97,  23,  17,  69,  21,
        43,  89, 276,  92, 183,   2, 132,  34, 131,   9,  78,  99, 129,
       216,   8,  29, 289,  30, 119, 228, 168, 208,  81, 147, 125,  94,
        51, 211,  64, 111, 266,  75, 306,  28, 232, 117, 118, 309, 303,
        98, 191, 116,  76, 113, 292,  88, 166, 244, 176, 258, 203, 231,
       142,  33, 157, 165,  50, 210, 294, 301, 106,  20, 318, 229, 204,
       269, 241, 178, 115, 174, 192,  67, 100, 141, 282, 122, 15

In [12]:
data_lite['NewExist'].unique()

#relever les lignes avec la valeur 0 de ma colonne
rows_with_zero = data_lite.loc[data_lite['NewExist']==0]
rows_with_zero

#remplacer les 0 par des Nan
data_lite['NewExist'] = data_lite['NewExist'].replace(0, np.nan)
data_lite['NewExist'].unique()

array([ 2.,  1., nan])

In [13]:
#verifier les valeurs uniques contenues par franchise code
data_lite['FranchiseCode'].unique()

array([    1,     0, 15100, ...,  2899, 18701, 15930])

In [14]:
data_lite['UrbanRural'].unique() #remplace 0 par NaN
dfur= data_lite.groupby(['UrbanRural'])['UrbanRural'].count()
dfur #plus de 320000 lignes ont une valeur urbanrural absente

#remplacer les 0 par des Nan
data_lite['UrbanRural'] = data_lite['UrbanRural'].replace(0, np.nan)
data_lite['UrbanRural'].unique()

array([nan,  1.,  2.])

In [15]:
#on fait de même ici
data_lite['LowDoc'].unique() #renvoie: ['Y', 'N', 'C', '1', nan, 'S', 'R', 'A', '0']
valeurs_odd = ['C', '1','S', 'R', 'A', '0']

data_lite['LowDoc'] = data_lite['LowDoc'].replace(valeurs_odd, np.nan)
data_lite['LowDoc'].unique()

array(['Y', 'N', nan], dtype=object)

In [16]:
data_lite.dtypes

LoanNr_ChkDgt          int64
Name                  object
City                  object
State                 object
Zip                    int64
Bank                  object
BankState             object
NAICS                  int64
ApprovalDate          object
ApprovalFY            object
Term                   int64
NoEmp                  int64
NewExist             float64
CreateJob              int64
RetainedJob            int64
FranchiseCode          int64
UrbanRural           float64
RevLineCr             object
LowDoc                object
ChgOffDate            object
DisbursementDate      object
DisbursementGross     object
BalanceGross          object
MIS_Status             int64
ChgOffPrinGr          object
GrAppv                object
SBA_Appv              object
dtype: object

In [17]:
#enlever le symbole $ de grappv et transforme grappv en float

data_lite["GrAppv"] = data_lite["GrAppv"].str.replace("$", "").str.replace(",", "").str.strip()
data_lite["GrAppv"] = data_lite["GrAppv"].astype(float)
data_lite.head(2)
data_lite.dtypes

/tmp/ipykernel_40092/1639391177.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_lite["GrAppv"] = data_lite["GrAppv"].str.replace("$", "").str.replace(",", "").str.strip()


LoanNr_ChkDgt          int64
Name                  object
City                  object
State                 object
Zip                    int64
Bank                  object
BankState             object
NAICS                  int64
ApprovalDate          object
ApprovalFY            object
Term                   int64
NoEmp                  int64
NewExist             float64
CreateJob              int64
RetainedJob            int64
FranchiseCode          int64
UrbanRural           float64
RevLineCr             object
LowDoc                object
ChgOffDate            object
DisbursementDate      object
DisbursementGross     object
BalanceGross          object
MIS_Status             int64
ChgOffPrinGr          object
GrAppv               float64
SBA_Appv              object
dtype: object

In [18]:
#la franchise code assigne un code lorsque l'entreprise est une franchise, ou 1 ou 0 lorsque ce n'est pas une franchise
# on peut remplacer les valeurs par '0': pas de franchise et '1': franchise en 2 étapes

data_lite.loc[data_lite['FranchiseCode']  <=1, 'FranchiseCode'] = 0

data_lite.loc[data_lite['FranchiseCode'] > 0, 'FranchiseCode'] = 1

data_lite['FranchiseCode'].unique()

array([0, 1])

In [19]:
#supprimer les ',' dans la colonne SBA_Appv
data_lite['SBA_Appv'] = data_lite['SBA_Appv'].str.strip()
data_lite['SBA_Appv'] = data_lite['SBA_Appv'].str.replace('$', '').str.replace(',', '')
data_lite['SBA_Appv'] = data_lite['SBA_Appv'].astype(float)

data_lite

/tmp/ipykernel_40092/3164089435.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_lite['SBA_Appv'] = data_lite['SBA_Appv'].str.replace('$', '').str.replace(',', '')


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28/02/1997,NaN,...,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,0,$0.00,60000.0,48000.0
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28/02/1997,NaN,...,N,Y,NaN,31-May-97,"$40,000.00",$0.00,0,$0.00,40000.0,32000.0
2,1000034009,"WHITLOCK DDS, TODD M.",BLOOMINGTON,IN,47401,GRANT COUNTY STATE BANK,IN,621210,28/02/1997,NaN,...,N,N,NaN,31-Dec-97,"$287,000.00",$0.00,0,$0.00,287000.0,215250.0
3,1000044001,"BIG BUCKS PAWN & JEWELRY, LLC",BROKEN ARROW,OK,74012,1ST NATL BK & TR CO OF BROKEN,OK,0,28/02/1997,NaN,...,N,Y,NaN,30-Jun-97,"$35,000.00",$0.00,0,$0.00,35000.0,28000.0
4,1000054004,"ANASTASIA CONFECTIONS, INC.",ORLANDO,FL,32801,FLORIDA BUS. DEVEL CORP,FL,0,28/02/1997,NaN,...,N,N,NaN,14-May-97,"$229,000.00",$0.00,0,$0.00,229000.0,229000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899159,9995573004,FABRIC FARMS,UPPER ARLINGTON,OH,43221,JPMORGAN CHASE BANK NATL ASSOC,IL,451120,27/02/1997,NaN,...,NaN,N,NaN,30-Sep-97,"$70,000.00",$0.00,0,$0.00,70000.0,56000.0
899160,9995603000,FABRIC FARMS,COLUMBUS,OH,43221,JPMORGAN CHASE BANK NATL ASSOC,IL,451130,27/02/1997,NaN,...,Y,N,NaN,31-Oct-97,"$85,000.00",$0.00,0,$0.00,85000.0,42500.0
899161,9995613003,"RADCO MANUFACTURING CO.,INC.",SANTA MARIA,CA,93455,"RABOBANK, NATIONAL ASSOCIATION",CA,332321,27/02/1997,NaN,...,N,N,NaN,30-Sep-97,"$300,000.00",$0.00,0,$0.00,300000.0,225000.0
899162,9995973006,"MARUTAMA HAWAII, INC.",HONOLULU,HI,96830,BANK OF HAWAII,HI,0,27/02/1997,NaN,...,N,Y,8-Mar-00,31-Mar-97,"$75,000.00",$0.00,1,"$46,383.00",75000.0,60000.0


In [20]:
# convertir 'ChgOffPrinGr', 'DisbursementGross' et 'BalanceGross', 'GrAppv' en float ou int
data_lite['ChgOffPrinGr'] = data_lite['ChgOffPrinGr'].str.replace('$', '').str.replace(',', '').astype(float)

data_lite['DisbursementGross'] = data_lite['DisbursementGross'].str.replace('$', '').str.replace(',', '').astype(float)

data_lite['BalanceGross'] = data_lite['BalanceGross'].str.replace('$', '').str.replace(',', '').astype(float)

data_lite[['ChgOffPrinGr', 'DisbursementGross', 'BalanceGross', 'GrAppv']].dtypes

/tmp/ipykernel_40092/3863240459.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_lite['ChgOffPrinGr'] = data_lite['ChgOffPrinGr'].str.replace('$', '').str.replace(',', '').astype(float)
/tmp/ipykernel_40092/3863240459.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_lite['DisbursementGross'] = data_lite['DisbursementGross'].str.replace('$', '').str.replace(',', '').astype(float)
/tmp/ipykernel_40092/3863240459.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data_lite['BalanceGross'] = data_lite['Balan

ChgOffPrinGr         float64
DisbursementGross    float64
BalanceGross         float64
GrAppv               float64
dtype: object

#### Verifier qu'il n'y a pas de lignes avec un comportement étrange en particulier MIS_Status 'PIF' associé à 'BalanceGross' non nul.

In [21]:
# on construit un df avec MIS_Status et 'ChgOffPrinGr' 
df_mc = data_lite[['MIS_Status', 'BalanceGross', 'DisbursementGross', 'SBA_Appv', 'GrAppv']]
df_mc

# on vérifie si il existe des lignes où MIS_Status est égal à 'P I F' et BalanceGross est supérieur à zéro
odd_rows = df_mc.loc[(df_mc['MIS_Status'] == 'P I F') & (df_mc['BalanceGross'] > 0)]

print(len(odd_rows))

#il il y a 14 lignes au total, on les supprime
odd_rows_indices = df_mc[(df_mc['MIS_Status'] == 'P I F') & (df_mc['BalanceGross'] > 0)].index

data_lite = data_lite.drop(index=odd_rows_indices)

# on vérifie que les lignes ont été supprimées
odd_rows_ = data_lite.loc[(data_lite['MIS_Status'] == 'P I F') & (data_lite['BalanceGross'] > 0)]

print(len(odd_rows_))

0
0


In [22]:
# Utilisez l'attribut shape pour obtenir le nombre de lignes et de colonnes de votre dataframe
nombre_lignes, nombre_colonnes = data_lite.shape

print("Nombre de lignes :", nombre_lignes)
print("Nombre de colonnes :", nombre_colonnes)


Nombre de lignes : 897167
Nombre de colonnes : 27


#### Enregistre le dataframe data_lite pour le réutiliser dans le notebook modelisation


In [23]:
data_lite.to_csv('data.csv', index=False)

